Notebook to evaluate overlap between LLM generated gene sets and

Outputs:
outputs/gen_time.tsv (time required to generate gene sts)
outputs/tok_use.tsv (table with tokens used for model/prompt combinations)
outputs/genes_overlap.tsv (overrepresentation results for LLM generated gene sets)

In [42]:
import pandas as pd
import json
from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests
import llm2geneset

In [43]:
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", 
             "WikiPathway_2023_Human"]
models = ["gpt-4o-mini-2024-07-18", "gpt-3.5-turbo-0125", "gpt-4o-2024-08-06"]
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", 
             "WikiPathway_2023_Human", "GO_Biological_Process_2023_sample1000"]
gene_sets = ["llm_genes_norole", "llm_genes_role", "llm_genes_reason",
             "llm_genes_conf_high", "llm_ensembled", "llm_genes_conf_low",
             "llm_genes_conf_medium"]

In [44]:
# Create gene sets with only high or high/medium confidence.
for model in models:
    for lib_name in lib_names:
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)

        llm_genes_conf_high = llm2geneset.sel_conf(gen_res["descr"], gen_res["llm_genes_conf"], ["high"])
        llm_genes_conf_medium = llm2geneset.sel_conf(gen_res["descr"], gen_res["llm_genes_conf"], ["medium"])
        llm_genes_conf_low = llm2geneset.sel_conf(gen_res["descr"], gen_res["llm_genes_conf"], ["low"])
        llm_genes_conf_high_medium = llm2geneset.sel_conf(gen_res["descr"], gen_res["llm_genes_conf"], ["high", "medium"])

        gen_res["llm_genes_conf_high"] = llm_genes_conf_high
        gen_res["llm_genes_conf_high_medium"] = llm_genes_conf_high_medium
        gen_res["llm_genes_conf_medium"] = llm_genes_conf_medium
        gen_res["llm_genes_conf_low"] = llm_genes_conf_low
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4) 

In [45]:
# Use ensemble generations to define gene sets.
for model in models:
    for lib_name in lib_names:
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)

        # Append separate generations together.
        llm_genes = [gen_res["llm_genes_norole"]]
        for i in range(4):
            llm_genes.append(gen_res["llm_ensemble_" + str(i)])
            
        # Generate ensembled set. 
        llm_ensembled = llm2geneset.ensemble_genes(gen_res["descr_cleaned"], llm_genes, 5)
    
        # Add ensembled results.
        gen_res["llm_ensembled"] = llm_ensembled
        
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)

In [46]:
# Output table with generation times.
gen_time_table = []
for lib_name in lib_names:    
    for model in models:
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
        x = {"lib_name": lib_name,
             "model": model,
             "gen_time_role": gen_res["gen_time_role"],
             "gen_time_norole": gen_res["gen_time_norole"],
             "gen_time_reasoning": gen_res["gen_time_reasoning"],
             "gen_time_conf": gen_res["gen_time_conf"]
        }
        gen_time_table.append(x)
df = pd.DataFrame(gen_time_table)
df.to_csv("outputs/gen_time.tsv",sep="\t",index=None)
df

,lib_name,model,gen_time_role,gen_time_norole,gen_time_reasoning,gen_time_conf
0,KEGG_2021_Human,gpt-4o-mini-2024-07-18,26.515748,15.174009,34.608303,24.790152
1,KEGG_2021_Human,gpt-3.5-turbo-0125,40.711580,21.634682,16.923785,14.943095
2,KEGG_2021_Human,gpt-4o-2024-08-06,32.681910,30.819701,39.722813,34.778510
3,Reactome_2022,gpt-4o-mini-2024-07-18,411.468713,50.428429,51.160516,801.404620
4,Reactome_2022,gpt-3.5-turbo-0125,35.365794,34.001247,43.410815,31.869861
5,Reactome_2022,gpt-4o-2024-08-06,93.410272,115.903737,52.931276,409.171145
6,WikiPathway_2023_Human,gpt-4o-mini-2024-07-18,401.165772,31.707753,58.294887,401.682324
7,WikiPathway_2023_Human,gpt-3.5-turbo-0125,98.878500,28.196767,19.184499,31.967324
8,WikiPathway_2023_Human,gpt-4o-2024-08-06,44.804884,33.037866,407.042043,41.880466
9,GO_Biological_Process_2023_sample1000,gpt-4o-mini-2024-07-18,68.248101,59.709785,145.046667,68.718979


In [47]:
hgnc_symbols = pd.read_csv("outputs/hgnc_symbols.txt", sep="\t", header=None)[0].tolist()
hgnc_symbols = set(hgnc_symbols)

In [48]:
gene_sets

['llm_genes_norole',
 'llm_genes_role',
 'llm_genes_reason',
 'llm_genes_conf_high',
 'llm_ensembled',
 'llm_genes_conf_low',
 'llm_genes_conf_medium']

In [49]:
database_res = []
tok_use = []

for lib_name in lib_names:
    print(lib_name)
    for model in models:
        print(model)
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
            
        for gset in gene_sets:
            print(gset)
            iou_output = []
            in_toks = 0
            out_toks = 0
            for idx in range(len(gen_res["descr"])):
                curated_genes = gen_res["curated_genesets"][idx]
                parsed_llm_genes = gen_res[gset][idx]['parsed_genes']
                ntries = gen_res[gset][idx]['ntries']
                
                in_toks += gen_res[gset][idx]['in_toks']
                out_toks += gen_res[gset][idx]['out_toks']
    
                non_hgnc = len(set(parsed_llm_genes) - hgnc_symbols)
    
                llm_genes = list(set(parsed_llm_genes)) # make sure unique genes are selected
    
            
                intersection = set(llm_genes).intersection(set(curated_genes))
                p_val = hypergeom.sf(len(intersection)-1,
                                     19846, 
                                     len(curated_genes), 
                                     len(llm_genes))
                
                # generatio == recall 
                generatio = float(len(intersection)) / len(set(curated_genes))                                                                                                 
                bgratio = float(len(set(llm_genes))) / 19846                                                                                                    
                                                                                                                                                                   
                richFactor = None                                                                                                                                      
                foldEnrich = None                                                                                                                                      
                if len(llm_genes) > 0:   
                    # richFactor == precision                                                                                                                              
                    richFactor = float(len(intersection)) / len(set(llm_genes))                                                                                        
                    foldEnrich = generatio / bgratio                                                                                                                   
  
                x = {
                    'database': lib_name,
                    'model' : model,
                    'gene_set' : gset, 
                    'descr': gen_res["descr"][idx],
                    'descr_cleaned': gen_res["descr_cleaned"][idx],
                    'ncurated': len(curated_genes),
                    'nllm': len(llm_genes),
                    'ninter': len(intersection),
                    'generatio': generatio,
                    'bgratio': bgratio,
                    'richFactor': richFactor,
                    'foldEnrich': foldEnrich,
                    'non_hgnc': non_hgnc,
                    'ntries' : ntries,
                    'ndup': len(parsed_llm_genes) - len(llm_genes),
                    'p_val': p_val
                }
                iou_output.append(x)

            toks_cur = {"lib_name": lib_name, "model": model,
                        "gene_set": gset,
                        "in_toks": in_toks, "out_toks" : out_toks}
            tok_use.append(toks_cur)
            df = pd.DataFrame(iou_output)
           
            _, pvals_corrected, _, _ = multipletests(df['p_val'], method='bonferroni')
            df['p_val_adj'] = pvals_corrected
            #& (df["generatio"] >= 0.5
            print(df[(df['p_val_adj'] < 0.01) ].shape[0] / df.shape[0])
            database_res.append(df)    
        print("")

KEGG_2021_Human
gpt-4o-mini-2024-07-18
llm_genes_norole
0.8625
llm_genes_role
0.840625
llm_genes_reason
0.803125
llm_genes_conf_high
0.709375
llm_ensembled
0.653125
llm_genes_conf_low
0.0125
llm_genes_conf_medium
0.409375

gpt-3.5-turbo-0125
llm_genes_norole
0.89375
llm_genes_role
0.890625
llm_genes_reason
0.74375
llm_genes_conf_high
0.6
llm_ensembled
0.59375
llm_genes_conf_low
0.046875
llm_genes_conf_medium
0.403125

gpt-4o-2024-08-06
llm_genes_norole
0.91875
llm_genes_role
0.94375
llm_genes_reason
0.903125
llm_genes_conf_high
0.815625
llm_ensembled
0.825
llm_genes_conf_low
0.10625
llm_genes_conf_medium
0.6625

Reactome_2022
gpt-4o-mini-2024-07-18
llm_genes_norole
0.6600660066006601
llm_genes_role
0.6716171617161716
llm_genes_reason
0.5654565456545655
llm_genes_conf_high
0.5099009900990099
llm_ensembled
0.4680968096809681
llm_genes_conf_low
0.015401540154015401
llm_genes_conf_medium
0.22167216721672167

gpt-3.5-turbo-0125
llm_genes_norole
0.6556655665566556
llm_genes_role
0.6606160616

In [9]:
df_tok = pd.DataFrame(tok_use)
df_tok.to_csv("outputs/tok_use.tsv", sep="\t", index=None)
df_tok[df_tok["model"] == "gpt-4o-2024-08-06"]["in_toks"].sum()/1e6*2.5+df_tok[df_tok["model"] == "gpt-4o-2024-08-06"]["out_toks"].sum()/1e6*10

138.1039225

In [10]:
df = pd.concat(database_res, ignore_index=True)

In [11]:
df

,database,model,gene_set,descr,descr_cleaned,ncurated,nllm,ninter,generatio,bgratio,richFactor,foldEnrich,non_hgnc,ntries,ndup,p_val,p_val_adj
0,KEGG_2021_Human,gpt-4o-mini-2024-07-18,llm_genes_norole,ABC transporters,ABC transporters,45,34,28,0.622222,0.001713,0.823529,363.194771,4,1,0,2.121814e-73,6.789806e-71
1,KEGG_2021_Human,gpt-4o-mini-2024-07-18,llm_genes_norole,AGE-RAGE signaling pathway in diabetic complic...,AGE-RAGE signaling pathway in diabetic complic...,100,17,9,0.090000,0.000857,0.529412,105.067059,1,1,0,3.405400e-17,1.089728e-14
2,KEGG_2021_Human,gpt-4o-mini-2024-07-18,llm_genes_norole,AMPK signaling pathway,AMPK signaling pathway,120,25,18,0.150000,0.001260,0.720000,119.076000,2,1,0,1.427988e-35,4.569563e-33
3,KEGG_2021_Human,gpt-4o-mini-2024-07-18,llm_genes_norole,Acute myeloid leukemia,Acute myeloid leukemia,67,20,7,0.104478,0.001008,0.350000,103.673134,1,1,0,2.710042e-13,8.672134e-11
4,KEGG_2021_Human,gpt-4o-mini-2024-07-18,llm_genes_norole,Adherens junction,Adherens junction,71,21,6,0.084507,0.001058,0.285714,79.863179,2,1,0,8.786538e-11,2.811692e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82714,GO_Biological_Process_2023_sample1000,gpt-4o-2024-08-06,llm_genes_conf_medium,Regulation Of Response To Biotic Stimulus (GO:...,Regulation Of Response To Biotic Stimulus,31,6,0,0.000000,0.000302,0.000000,0.000000,0,1,0,1.000000e+00,1.000000e+00
82715,GO_Biological_Process_2023_sample1000,gpt-4o-2024-08-06,llm_genes_conf_medium,Cellular Response To Catecholamine Stimulus (G...,Cellular Response To Catecholamine Stimulus,19,8,0,0.000000,0.000403,0.000000,0.000000,0,1,0,1.000000e+00,1.000000e+00
82716,GO_Biological_Process_2023_sample1000,gpt-4o-2024-08-06,llm_genes_conf_medium,Positive Regulation Of Natural Killer Cell Cyt...,Positive Regulation Of Natural Killer Cell Cyt...,6,7,0,0.000000,0.000353,0.000000,0.000000,0,1,0,1.000000e+00,1.000000e+00
82717,GO_Biological_Process_2023_sample1000,gpt-4o-2024-08-06,llm_genes_conf_medium,Negative Regulation Of Cardiac Muscle Hypertro...,Negative Regulation Of Cardiac Muscle Hypertrophy,12,4,0,0.000000,0.000202,0.000000,0.000000,1,1,0,1.000000e+00,1.000000e+00


In [11]:
df.to_csv("outputs/genes_overlap.tsv",sep="\t",index=None)

Reviewer suggest analysis on overlap between ensembled and high confidence genes.

In [ ]:


iou_output = [ ]
for lib_name in lib_names:
    print(lib_name)
    for model in models:
        print(model)
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
            
        gset1 = "llm_genes_conf_high"
        gset2 = "llm_ensembled"
     
        for idx in range(len(gen_res["descr"])):
            parsed_llm_genes1 = gen_res[gset1][idx]['parsed_genes']
            parsed_llm_genes2 = gen_res[gset2][idx]['parsed_genes']
            intersection = set(parsed_llm_genes1).intersection(set(parsed_llm_genes2))
            union = set(parsed_llm_genes1).union(set(parsed_llm_genes2))
            jaccard_similarity = len(intersection) / len(union) if len(union) > 0 else 0
            x = {
                'database': lib_name,
                'model' : model,
                'descr': gen_res["descr"][idx],
                'descr_cleaned': gen_res["descr_cleaned"][idx],
                'ninter': len(intersection),
                'nunion': len(union),
                'jaccard_similarity': jaccard_similarity    
            }   
            iou_output.append(x)         

pd.DataFrame(iou_output).to_csv("outputs/overlap_high_vs_ensemble.tsv",sep="\t",index=None)

KEGG_2021_Human
gpt-4o-mini-2024-07-18
gpt-3.5-turbo-0125
gpt-4o-2024-08-06
Reactome_2022
gpt-4o-mini-2024-07-18
gpt-3.5-turbo-0125
gpt-4o-2024-08-06
WikiPathway_2023_Human
gpt-4o-mini-2024-07-18
gpt-3.5-turbo-0125
gpt-4o-2024-08-06
GO_Biological_Process_2023_sample1000
gpt-4o-mini-2024-07-18
gpt-3.5-turbo-0125
gpt-4o-2024-08-06


,database,model,descr,descr_cleaned,ninter,nunion,jaccard_similarity
0,KEGG_2021_Human,gpt-4o-mini-2024-07-18,ABC transporters,ABC transporters,13,18,0.722222
1,KEGG_2021_Human,gpt-4o-mini-2024-07-18,AGE-RAGE signaling pathway in diabetic complic...,AGE-RAGE signaling pathway in diabetic complic...,1,4,0.250000
2,KEGG_2021_Human,gpt-4o-mini-2024-07-18,AMPK signaling pathway,AMPK signaling pathway,3,12,0.250000
3,KEGG_2021_Human,gpt-4o-mini-2024-07-18,Acute myeloid leukemia,Acute myeloid leukemia,5,10,0.500000
4,KEGG_2021_Human,gpt-4o-mini-2024-07-18,Adherens junction,Adherens junction,1,3,0.333333
...,...,...,...,...,...,...,...
11812,GO_Biological_Process_2023_sample1000,gpt-4o-2024-08-06,Regulation Of Response To Biotic Stimulus (GO:...,Regulation Of Response To Biotic Stimulus,2,6,0.333333
11813,GO_Biological_Process_2023_sample1000,gpt-4o-2024-08-06,Cellular Response To Catecholamine Stimulus (G...,Cellular Response To Catecholamine Stimulus,3,12,0.250000
11814,GO_Biological_Process_2023_sample1000,gpt-4o-2024-08-06,Positive Regulation Of Natural Killer Cell Cyt...,Positive Regulation Of Natural Killer Cell Cyt...,4,5,0.800000
11815,GO_Biological_Process_2023_sample1000,gpt-4o-2024-08-06,Negative Regulation Of Cardiac Muscle Hypertro...,Negative Regulation Of Cardiac Muscle Hypertrophy,0,2,0.000000
